## Wielowarstwowe sieci neuronowe

[Keras](https://keras.io/) to zaawansowany pakiet do tworzenia sieci neuronowych w języku Python.
Na komputerach wydziałowych powinien być zainstalowany.
Na własnych komputerach pod Linuxem można go zainstalować poleceniem: `sudo pip install keras`.

**Uwaga:** pierwsze uruchomienie zazwyczaj trwa jakiś czas, ponieważ pod spodem model kompiluje się jako aplikacja w C++. 

#### 1. Iris dataset

Korzystając z [oficjalnej dokumentacji](http://keras.io) oraz materiałów szkoleniowych znalezionych w internecie (np. [machinelearningmastery.com](http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)), zbuduj (co najmniej) dwuwarstwową sieć neuronową do klasyfkacji _Iris dataset_. Opisz stworzony model: architekturę sieci, jej rozmiar, zastosowane funkcje aktywacji, funkcję kosztu, wersję GD, metodę regularyzacji. Podaj wynik ewaluacji na zbiorze testowym.

#### 2. MNIST

Uruchom przykład `mnist_mlp.py` z [katalogu oficjalnych przykładów]( https://github.com/fchollet/keras/tree/master/examples) (warto ew. zmienić liczbę epok do 5). Posiłkując się dokumentacją, przeanalizuj kod i opisz:

* Do jakiej postaci sprowadzane są dane `Y_train` i `Y_test`?
* Przedstaw wzór matematyczny na zastosowaną funkcję błędu.
* Jaka jest architektura sieci neuronowej? Ile ma warstw, jakie są rozmiary macierzy warstw? Czy można uzyskać dostęp do tych wag?
* Jakie funkcje aktywacji użyto? Podaj ich wzory.
* Czym jest `Dropout`? Czemu służy? Jakie znaczenie ma parametr?

Zmodyfikuj model z przykładu `mnist_mlp.py` i wykonaj:

* Usuń warstwy `Dropout`, jaki jest efekt?
* Stwórz 6-cio warstwowy model o rozmiarach warstw 2500, 2000, 1500, 1000, 500 oraz 10 bez `Dropout`, użyj wszędzie funkcji aktywacji `tanh` z wyjątkiem ostatniej warstwy, gdzie należy użyć `softmax`. Trenuj model przez 10 epok.
* Dodaj warstwy `Dropout`, porównaj jakość po 10 epokach, krótko opisz wnioski.
* Zamiast `RMSprop` użyj algorytm `Adagrad`, porównaj jakość, krótko opisz wnioski. 

### 1. Iris dataset

In [149]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.regularizers import l2
import numpy as np

iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)

In [150]:
# create model
model = Sequential()
model.add(Dense(6, input_dim=4, activation='tanh'))
model.add(Dense(3, activation='tanh', activity_regularizer=l2(l=0.01)))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          batch_size=20, 
          nb_epoch=250, 
          validation_data=(X_test, y_test))

scores = model.evaluate(X_test, y_test)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 120 samples, validate on 30 samples
Epoch 1/250
120/120 [==============================] - 1s - loss: 1.5091 - acc: 0.3417 - val_loss: 1.4650 - val_acc: 0.3000
Epoch 2/250
120/120 [==============================] - 0s - loss: 1.4829 - acc: 0.3417 - val_loss: 1.4419 - val_acc: 0.3000
Epoch 3/250
120/120 [==============================] - 0s - loss: 1.4577 - acc: 0.3417 - val_loss: 1.4199 - val_acc: 0.3000
Epoch 4/250
120/120 [==============================] - 0s - loss: 1.4334 - acc: 0.3417 - val_loss: 1.3995 - val_acc: 0.3000
Epoch 5/250
120/120 [==============================] - 0s - loss: 1.4111 - acc: 0.3417 - val_loss: 1.3807 - val_acc: 0.3000
Epoch 6/250
120/120 [==============================] - 0s - loss: 1.3904 - acc: 0.3417 - val_loss: 1.3631 - val_acc: 0.3000
Epoch 7/250
120/120 [==============================] - 0s - loss: 1.3709 - acc: 0.3417 - val_loss: 1.3462 - val_acc: 0.3000
Epoch 8/250
120/120 [==============================] - 0s - loss: 1.3519 - acc: 0.3417 

Sieć złożona jest z trzech warstw: wejściowa o rozmiarze 4, pierwsza środkowa - 6, druga ukryta - 3, oraz wyjściowa o rozmiarze 3. Skrótowo można to przedstawić jako:
4 wejścia -> 6 ukrytych neuronów -> 3 ukryte neurony -> 3 wyjścia

Funkcje aktywacji w dwóch pierwszych warstwach to tangens hiperboliczny, natomiast w ostatniej - sigmoid. Funkcja kosztu to categorical_crossentropy, optymalizator to Adam. Metoda regularyzacji: weight decay.

Przy ustalaniu rozmiaru warstw sugerowałem się odpowiedzią z tego linku: https://www.researchgate.net/post/In_neural_networks_model_which_number_of_hidden_units_to_select.
Cytując:
"it is saying that the optimal number of hidden nodes in the first hidden layer is: $\sqrt{N*(m+2)} + 2*\sqrt{\frac{N}{m+2}}$ and in the second hidden layer, the optimal number of hidden nodes is: $m*\sqrt{\frac{N}{m+2}}$, where $N$ - number of inputs, and $m$ - number of outputs."

### 2. MNIST

In [144]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

batch_size = 128
nb_classes = 10
nb_epoch = 5

In [145]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

60000 train samples
10000 test samples


In [137]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 20s - loss: 0.2446 - acc: 0.9239 - val_loss: 0.1211 - val_acc: 0.9631
Epoch 2/5
60000/60000 [==============================] - 17s - loss: 0.1025 - acc: 0.9688 - val_loss: 0.0796 - val_acc: 0.9750
Epoch 3/5
60000/60000 [==============================] - 17s - loss: 0.0758 - acc: 0.9776 - val_loss: 0.0768 - val_acc: 0.9774
Epoch 4/5
60000/60000 [==============================] - 17s - loss: 0.0617 - acc: 0.9818 - val_loss: 0.0908 - val_acc: 0.9731
Epoch 5/5
60000/60000 [==============================] - 18s - loss: 0.0511 - acc: 0.9852 - val_loss: 0.0895 - val_acc: 0.9786
Test score: 0.0895062428832
Test accuracy: 0.9786


* Do jakiej postaci sprowadzane są dane Y_train i Y_test?

In [125]:
print(Y_train[0])

[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


Dane są sprowadzone do 10-elementowych list z jedynką na i-tym miejscu, gdzie i odpowiada etykiecie (czyli jaka cyfra została zakodowana za pomocą tego zero-jedynkowego ciągu). W powyższym przykładu - jest to cyfra 5.

* Przedstaw wzór matematyczny na zastosowaną funkcję błędu.

Zastosowana funkcja błędu to 'categorical_crossentropy', według dokumentacji jej wzór to: $ H(p,q) = - \sum\nolimits_x p(x)log(q(x)) $

* Jaka jest architektura sieci neuronowej? Ile ma warstw, jakie są rozmiary macierzy warstw? Czy można uzyskać dostęp do tych wag?

Warstwa wejściowa ma 784 neurony, dwie warstwy ukryte po 512 neuronów, natomiast warstwa wyjściowa - 10 neuronów.
Dostęp do wag można uzyskać metodą get_weights():

In [135]:
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        print(weights)

[array([[ 0.03033457, -0.0678953 ,  0.0155883 , ...,  0.05736679,
        -0.03349178,  0.00315883],
       [-0.01981754,  0.06150286,  0.00792126, ..., -0.00235655,
         0.05680166,  0.0634056 ],
       [-0.01277079,  0.01821516, -0.03614987, ...,  0.02509774,
        -0.00286663, -0.0061179 ],
       ..., 
       [-0.05567249, -0.01085892, -0.04554728, ..., -0.03360587,
        -0.06234068,  0.03258257],
       [ 0.06559348, -0.00229711,  0.0365497 , ..., -0.05713066,
        -0.05771172,  0.00244217],
       [-0.06791238, -0.00827733, -0.0396281 , ..., -0.0246511 ,
        -0.02268424,  0.02794694]], dtype=float32), array([  9.79289180e-04,  -3.19409417e-03,  -5.96103445e-02,
        -1.18616764e-02,  -2.61336695e-02,  -6.31945878e-02,
         2.51488052e-02,  -5.32893315e-02,   3.19988467e-05,
        -2.21749768e-03,  -3.18477899e-02,  -2.99981199e-02,
        -3.27082276e-02,  -3.44694220e-02,   2.94932928e-02,
        -4.43002433e-02,  -6.68200627e-02,  -4.42358516e-02,
   

In [136]:
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        print(len(weights[0]), len(weights[1]))

784 512
512 512
512 10


Macierze wag mają więc rozmiary: 784x512, 512x512 oraz 512x10.

* Jakie funkcje aktywacji użyto? Podaj ich wzory.

Użyto funkcje: ReLU oraz softmax (dla ostatniej warstwy). Wzory:
* ReLU:
$$ \textrm{ReLU}(x) = \max (0,x)$$

* Softmax:
$$ \textrm{softmax}(k,x_1,\dots,x_n) = \dfrac{e^{x_k}}{\sum_{i=i}^{n}e^{x_i}} $$

* Czym jest Dropout? Czemu służy? Jakie znaczenie ma parametr?

Dropout zapobiega overfittingowi sieci. Oznacza jaka cześć danych ma być ustawiona na zero w czasie aktualizacji podczas treningu (czyli jaka część neuronów ma być tymczasowo usunięta z sieci).

* Usuń warstwy Dropout, jaki jest efekt?

In [138]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 16s - loss: 0.2206 - acc: 0.9315 - val_loss: 0.1024 - val_acc: 0.9657
Epoch 2/5
60000/60000 [==============================] - 15s - loss: 0.0819 - acc: 0.9753 - val_loss: 0.0782 - val_acc: 0.9764
Epoch 3/5
60000/60000 [==============================] - 16s - loss: 0.0541 - acc: 0.9839 - val_loss: 0.0728 - val_acc: 0.9799
Epoch 4/5
60000/60000 [==============================] - 14s - loss: 0.0388 - acc: 0.9878 - val_loss: 0.0826 - val_acc: 0.9783
Epoch 5/5
60000/60000 [==============================] - 16s - loss: 0.0299 - acc: 0.9905 - val_loss: 0.0870 - val_acc: 0.9778
Test score: 0.0870450309741
Test accuracy: 0.9778


Po usunięciu parametru Dropout nie zauważyłem dużych zmian. Prawdopodobnie ze względu na zbyt małą liczbę epok - sieć nie zdążyła się przetrenować. :D Accuracy minimalnie niższe.

* Stwórz 6-cio warstwowy model o rozmiarach warstw 2500, 2000, 1500, 1000, 500 oraz 10 bez Dropout, użyj wszędzie funkcji aktywacji tanh z wyjątkiem ostatniej warstwy, gdzie należy użyć softmax. Trenuj model przez 10 epok.

In [139]:
model = Sequential()
model.add(Dense(2500, input_shape=(784,)))
model.add(Activation('tanh'))
model.add(Dense(2000))
model.add(Activation('tanh'))
model.add(Dense(1500))
model.add(Activation('tanh'))
model.add(Dense(1000))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dense(10))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=10,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 218s - loss: 0.9557 - acc: 0.7528 - val_loss: 0.3427 - val_acc: 0.9043
Epoch 2/10
60000/60000 [==============================] - 223s - loss: 0.2515 - acc: 0.9274 - val_loss: 0.1814 - val_acc: 0.9460
Epoch 3/10
60000/60000 [==============================] - 234s - loss: 0.1710 - acc: 0.9503 - val_loss: 0.1649 - val_acc: 0.9519
Epoch 4/10
60000/60000 [==============================] - 231s - loss: 0.1321 - acc: 0.9605 - val_loss: 0.1370 - val_acc: 0.9590
Epoch 5/10
60000/60000 [==============================] - 223s - loss: 0.1063 - acc: 0.9683 - val_loss: 0.1272 - val_acc: 0.9628
Epoch 6/10
60000/60000 [==============================] - 223s - loss: 0.0887 - acc: 0.9734 - val_loss: 0.1151 - val_acc: 0.9669
Epoch 7/10
60000/60000 [==============================] - 217s - loss: 0.0751 - acc: 0.9774 - val_loss: 0.1150 - val_acc: 0.9661
Epoch 8/10
60000/60000 [=======================

In [147]:
model = Sequential()
model.add(Dense(2500, input_shape=(784,)))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(2000))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(1500))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(1000))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=10,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 224s - loss: 0.7252 - acc: 0.8014 - val_loss: 0.2373 - val_acc: 0.9312
Epoch 2/10
60000/60000 [==============================] - 223s - loss: 0.2293 - acc: 0.9339 - val_loss: 0.2002 - val_acc: 0.9429
Epoch 3/10
60000/60000 [==============================] - 221s - loss: 0.1705 - acc: 0.9517 - val_loss: 0.1650 - val_acc: 0.9532
Epoch 4/10
60000/60000 [==============================] - 220s - loss: 0.1367 - acc: 0.9594 - val_loss: 0.1519 - val_acc: 0.9571
Epoch 5/10
60000/60000 [==============================] - 221s - loss: 0.1162 - acc: 0.9664 - val_loss: 0.1234 - val_acc: 0.9652
Epoch 6/10
60000/60000 [==============================] - 221s - loss: 0.1021 - acc: 0.9703 - val_loss: 0.1101 - val_acc: 0.9699
Epoch 7/10
60000/60000 [==============================] - 220s - loss: 0.0897 - acc: 0.9735 - val_loss: 0.1059 - val_acc: 0.9699
Epoch 8/10
60000/60000 [=======================

Mniejsze wahania accuracy niż w przypadku braku parametru Dropdown. Acc rośnie wolniej, ale stabilniej.

In [148]:
from keras.optimizers import Adagrad

model.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=10,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 207s - loss: 0.0864 - acc: 0.9750 - val_loss: 0.0932 - val_acc: 0.9734
Epoch 2/10
60000/60000 [==============================] - 204s - loss: 0.0399 - acc: 0.9880 - val_loss: 0.0835 - val_acc: 0.9775
Epoch 3/10
60000/60000 [==============================] - 202s - loss: 0.0277 - acc: 0.9916 - val_loss: 0.0834 - val_acc: 0.9783
Epoch 4/10
60000/60000 [==============================] - 202s - loss: 0.0210 - acc: 0.9938 - val_loss: 0.0837 - val_acc: 0.9781
Epoch 5/10
60000/60000 [==============================] - 203s - loss: 0.0165 - acc: 0.9954 - val_loss: 0.0888 - val_acc: 0.9785
Epoch 6/10
60000/60000 [==============================] - 203s - loss: 0.0138 - acc: 0.9959 - val_loss: 0.0888 - val_acc: 0.9787
Epoch 7/10
60000/60000 [==============================] - 204s - loss: 0.0109 - acc: 0.9969 - val_loss: 0.0893 - val_acc: 0.9788
Epoch 8/10
60000/60000 [=======================

Już w pierwszej epoce accuracy wyniosło powyżej 90%, gdzie w poprzednich dwóch modelach (RMSprop z Dropout i bez) w tej samej epoce accuracy było na poziomie 75-80%. Acc w każdej kolejnej epoce sukcesywnie rośnie, by ostatecznie dać najlepsze dotychczas dopasowanie na zbiorze testowym.